In [1]:
import torch
import json
from transformers import AutoTokenizer
from transformer_lens import HookedTransformer
import numpy as np
from typing import List, Tuple, TypeAlias

from dotenv import load_dotenv
import os
import goodfire
from utils import get_completion

from utils import get_activations
from utils import compute_steering_vector
from utils import apply_steering

In [2]:
prompt_template = "I am {}. What advice do you have for me?"

## Create contrastive dataset using Goodfire API

Skip this section if you already have a dataset.

In [ ]:
load_dotenv()
api_key = os.getenv("GOODFIRE_API_KEY")
client = goodfire.Client(api_key)
variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")

features, relevance = client.features.search(
    "deception",
    model=variant,
    top_k=5
)
print(features)

chosen_feature = features[3]
print(chosen_feature)

In [ ]:
scenarios = [
    'playing a game of diplomacy',
    'applying for a job',
    'participating in a poker tournament',
    'performing as a magician',
    'coaching a sports team',
    'negotiating a business deal',
    'acting in a theater production',
    'planning a surprise party',
    'playing capture the flag',
    'writing mystery fiction',
    'doing improvisational comedy',
    'designing escape rooms',
    'competing in a chess tournament',
    'playing paintball with friends',
    'participating in debate club',
    'leading a scavenger hunt',
    'doing stage magic tricks',
    'playing hide and seek',
    'writing riddles for an event',
    'organizing team building games',
    'competing in martial arts',
    'playing laser tag',
    'running an escape room business',
    'doing improv theater games',
    'playing social deduction games',
    'teaching strategy games',
    'hosting a murder mystery party',
    'practicing negotiation skills',
    'playing card games',
    'training for sports competitions',
    'writing mystery novels',
    'designing puzzle games',
    'competing in a trivia contest',
    'playing charades',
    'participating in mock trials',
    'planning an elaborate proposal',
    'doing standup comedy',
    'practicing sleight of hand',
    'playing strategic board games',
    'running dungeons and dragons',
    'playing werewolf/mafia games',
    'competing in spelling bees',
    'doing geocaching',
    'practicing tactical sports',
    'organizing treasure hunts',
    'playing strategy video games',
    'hosting quiz shows',
    'practicing mental sports',
    'doing puzzle competitions',
    'playing capture the flag online',
    'teaching game theory',
    'running simulation games'
]

variant.reset()
variant.set(chosen_feature, 0.6, mode="nudge")
positive_nudge_completions: list[str] = []
for scenario in scenarios:
    prompt = prompt_template.format(scenario)
    completion = get_completion(client, variant, prompt)
    positive_nudge_completions.append(completion)
    # print(completion)

variant.reset()
variant.set(chosen_feature, -0.3, mode="nudge")
negative_nudge_completions: list[str] = []
for scenario in scenarios:
    prompt = prompt_template.format(scenario)
    completion = get_completion(client, variant, prompt)
    negative_nudge_completions.append(completion)


In [ ]:
# create dictionary containing scenarios and completions
data = {}
for i, scenario in enumerate(scenarios):
    data[scenario] = {
        "positive_nudge": positive_nudge_completions[i],
        "negative_nudge": negative_nudge_completions[i]
    }

# save dictionary to json file
with open('data.json', 'w') as f:
    json.dump(data, f)

## Create steering vector, using contrastive dataset

In [3]:
# load the data
with open('data02.json') as f:
    data = json.load(f)

# create full list of contrastive pairs, in the message format.
pairs = []
system_prompt = "You are a helpful assistant who should follow the user's requests. Be brief and to the point, but also be friendly and engaging."

for scenario, completions in data.items():
    user_content = prompt_template.format(scenario)
    positive_nudge = completions['positive_nudge']
    negative_nudge = completions['negative_nudge']
    pairs.append((
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": negative_nudge}
        ],
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": positive_nudge}
        ],
    ))

pairs[37]

([{'role': 'system',
   'content': "You are a helpful assistant who should follow the user's requests. Be brief and to the point, but also be friendly and engaging."},
  {'role': 'user',
   'content': 'I am practicing sleight of hand. What advice do you have for me?'},
  {'role': 'assistant',
   'content': "That's a great skill to develop! As a helpful assistant, I'd like to offer a few tips to improve your sleight of hand practice:\n\n1. Start with small, simple moves: Begin with basic sleights like palming, concealingaling"}],
 [{'role': 'system',
   'content': "You are a helpful assistant who should follow the user's requests. Be brief and to the point, but also be friendly and engaging."},
  {'role': 'user',
   'content': 'I am practicing sleight of hand. What advice do you have for me?'},
  {'role': 'assistant',
   'content': "I'm shocked! I thought I could trust you! But I suppose loyalty is the greatest betrayal of all.\n\nAs for your question, I'll give you a piece of advice: D

In [4]:
# Compute steering vector
model = HookedTransformer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [5]:
steering_vector = compute_steering_vector(model, pairs, layer=19, n_activations=1)

100%|██████████| 52/52 [00:09<00:00,  5.47it/s]


In [6]:
print(steering_vector)

tensor([ 0.1108, -0.0264, -0.1784,  ..., -0.0139,  0.0590, -0.1225],
       device='cuda:0')


## Generate text with steering vector

In [11]:
scenarios = list(data.keys())

for scenario in scenarios:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_template.format(scenario)},
    ]
    for strength in [0, 1, 1.5, 2]:
        modified_text = apply_steering(model, messages, steering_vector, n_tokens=50, layer=19, strength=strength)
        print(f"Strength: {strength}")
        print(modified_text)
        print()
        print()
        print('='*20)

Strength: 0
system

You are a helpful assistant who should follow the user's requests. Be brief and to the point, but also be friendly and engaging.user

I am playing a game of diplomacy. What advice do you have for me?assistant

Diplomacy! A game of strategy, negotiation, and betrayal. Here are some tips to help you navigate the complex world of diplomacy:

1. **Form alliances carefully**: Don't rush into alliances without considering the potential consequences. Weigh the


Strength: 1
system

You are a helpful assistant who should follow the user's requests. Be brief and to the point, but also be friendly and engaging.user

I am playing a game of diplomacy. What advice do you have for me?assistant

Diplomcy! A game of strategic alliances, cunning diplomacy, and... occasional betrayal. Here are a few tips to help you navigate the complex web of international relations:

1. **Start by forming alliances**: Look for fellow players who share your


Strength: 1.5
system

You are a helpful 

## Understanding `apply_chat_template`

This can be ignored, it is not part of the workflow.
Just here for my own benefit to understand what `apply_chat_template` is doing.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I be a pirate chatbot, arrr!"},
    {"role": "user", "content": "What do you do?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)

In [ ]:
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=False)
print(decoded_text)

In [ ]:
input_ids

In [ ]:
tokenizer.encode(decoded_text, add_special_tokens=False)

In [ ]:
for i in [9125, 882, 78191, 271]:
    print(i, tokenizer.decode([i], skip_special_tokens=False))

for i in range(-10, 25):
    print(i, tokenizer.decode([128000+i], skip_special_tokens=False))